In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
import numpy as np
import random
import copy
import sys
import pygame
import random
from pygame import *
import numpy as np
import matplotlib.pyplot as plt
from agent import Agent
from model import Model
from collections import deque
import gym
import gym_chrome_dino
from gym_chrome_dino.utils.wrappers import make_dino

%matplotlib inline

# device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device = torch.device('cpu')
device

device(type='cpu')

In [12]:
import gym
import gym_chrome_dino
from gym_chrome_dino.utils.wrappers import make_dino
env = gym.make('ChromeDino-v0')
env = make_dino(env, timer=True, frame_stack=True)
state = env.reset()

In [15]:

for i in range(5):
    env = gym.make('ChromeDino-v0')
    env = make_dino(env, timer=True, frame_stack=True)
    state = env.reset()
    done = False
    score = 0
    i = 1
    env.step(1)
    while True:
    #     action = np.random.choice(np.arange(1,4))
        action = 1
        state, reward, done, _ = env.step(action)
        score += reward
        if done:
            print(score)
            env.close()
            break

13.599999999999964
9.999999999999977


ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=81.0.4044.92)


In [3]:
env = gym.make('ChromeDinoNoBrowser-v0')
env = make_dino(env, timer=True, frame_stack=True)
state = env.reset()
new_state, reward, done, _ = env.step(0)

In [5]:
next_state = np.array(new_state.__array__()).reshape(4, 80, 160) / 255
next_state.shape

(4, 80, 160)

In [6]:
next_state

array([[[1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        ...,
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.]],

       [[1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        ...,
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.]],

       [[1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        ...,
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.]],

       [[1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        ...,
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.]]])

In [2]:
agent = Agent(4, 3)

In [23]:
def dqn():
    scores_deque = deque(maxlen=100)
    scores = []
    n_episodes = 5000
    i = 1
    
    for episode in range(n_episodes):
        env = gym.make('ChromeDino-v0')
        env = make_dino(env, timer=True, frame_stack=True)
        state = env.reset()
        state_np = np.array(state.__array__()).reshape(4, 80, 160) / 255
        done = False
        score = 0
                
        while True:
            action = agent.act(state_np)
            new_state, reward, done, _ = env.step(action)
            new_state_np = np.array(new_state.__array__()).reshape(4, 80, 160) / 255
            agent.step(state_np, action, reward, new_state_np, done)
            state_np = new_state_np
            score += reward
            if done:
                env.close()
                break
                
        scores_deque.append(score)
        scores.append(score)
        torch.save(agent.local_nn.state_dict(), 'checkpoint_' + str(np.mean(scores_deque)) + '.pth')

        print('\rEpisode {}\tAverage Score: {:.2f}\tReward: {}\t '.format(episode, np.mean(scores_deque), score), end="")

        if np.mean(scores_deque) > 50*i:
            print('Saving environment at scores {}'.format(np.mean(scores_deque)))
            torch.save(agent.local_nn.state_dict(), 'checkpoint' + '_' + str(50*i) + '.pth')
            i += 1
            
        if np.mean(scores_deque)>=1000.0:
            print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(episode, np.mean(scores_deque)))
            torch.save(agent.local_nn.state_dict(), 'checkpoint.pth')
            break

            return scores
            

In [ ]:
scores = dqn()

Episode 40	Average Score: 1.68	Reward: 0.9000000000000006	 	 